In [1]:
import hail as hl
# !module load java/java-1.8
gs = "/data/scratch/gnomAD/v3/gnomad.genomes.v3.1.sites.ht/"
data = hl.read_table(gs)
data

Initializing Hail with default parameters...
Running on Apache Spark version 2.4.1
SparkUI available at http://vega.local:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /enadisk/maison/sage2016/weber/PycharmProjects/EXOTIC/notebooks/hail-20210301-1808-0.2.62-84fa81b9ea3d.log


In [21]:
bed_file = hl.import_bed('/gstock/EXOTIC/data/GENOMICS/BED_test.bed', reference_genome='GRCh38')
bed_file.show()

2021-03-01 19:17:38 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
  Loading field 'f3' as type str (user-supplied)
  Loading field 'f4' as type str (user-supplied)
  Loading field 'f5' as type str (not specified)


,
interval,target
interval<locus<GRCh38>>,str
[chr1:33302016-chr1:33302266),"""Upstream"""
[chr1:33302266-chr1:33302516),"""Upstream"""
[chr1:33302516-chr1:33302766),"""Upstream"""
[chr1:33302766-chr1:33303016),"""Upstream"""
[chr1:33303016-chr1:33303266),"""Upstream"""
[chr1:33303266-chr1:33303516),"""Upstream"""
[chr1:33303516-chr1:33303766),"""Upstream"""
[chr1:33303766-chr1:33304016),"""Upstream"""


In [28]:
bed_file = hl.import_table('/gstock/EXOTIC/data/GENOMICS/BED_test_h.bed')
bed_file.show()


2021-03-01 19:20:15 Hail: INFO: Reading table without type imputation
  Loading field 'CHR' as type str (not specified)
  Loading field 'Start' as type str (not specified)
  Loading field 'End' as type str (not specified)
  Loading field 'Region_type' as type str (not specified)
  Loading field 'Num' as type str (not specified)
  Loading field 'Gene' as type str (not specified)


,,,,,
CHR,Start,End,Region_type,Num,Gene
str,str,str,str,str,str
"""chr2""","""58345183""","""58345598""","""Gene_body""","""1""","""A1BG"""
"""chr2""","""58345598""","""58346013""","""Gene_body""","""2""","""A1BG"""
"""chr2""","""58346013""","""58346428""","""Gene_body""","""3""","""A1BG"""
"""chr2""","""58346428""","""58346843""","""Gene_body""","""4""","""A1BG"""
"""chr2""","""58346843""","""58347258""","""Gene_body""","""5""","""A1BG"""
"""chr2""","""58347258""","""58347673""","""Gene_body""","""6""","""A1BG"""
"""chr2""","""58347673""","""58348088""","""Gene_body""","""7""","""A1BG"""
"""chr2""","""58348088""","""58348503""","""Gene_body""","""8""","""A1BG"""


In [12]:
test = ['chr1:50799409-chr1:50885666']

intervals = bed_file.annotate(interval_copy = bed_file.interval)
intervals.show()

intervals_raw = [hl.parse_locus_interval('{}'.format(e), reference_genome='GRCh38') for e in test] # or some other way to make a list of
data_lite = hl.filter_intervals(data, intervals_raw)

data_lite = data_lite.filter((data_lite.popmax.AC > 0))
data_lite = data_lite.select(data_lite.popmax.AC, data_lite.popmax.AF, data_lite.vep, data_lite.cadd)

print(data_lite.count())

data_lite = data_lite.annotate(
    interval = intervals[data_lite.locus].interval_copy,
    gene = intervals[data_lite.locus].target,
)
data_lite.tail(1000).show()


2021-03-01 19:13:20 Hail: INFO: Ordering unsorted dataset with network shuffle


,,
interval,target,interval_copy
interval<locus<GRCh38>>,str,interval<locus<GRCh38>>
[chr1:33302016-chr1:33302266),"""Upstream""",[chr1:33302016-chr1:33302266)
[chr1:33302266-chr1:33302516),"""Upstream""",[chr1:33302266-chr1:33302516)
[chr1:33302516-chr1:33302766),"""Upstream""",[chr1:33302516-chr1:33302766)
[chr1:33302766-chr1:33303016),"""Upstream""",[chr1:33302766-chr1:33303016)
[chr1:33303016-chr1:33303266),"""Upstream""",[chr1:33303016-chr1:33303266)
[chr1:33303266-chr1:33303516),"""Upstream""",[chr1:33303266-chr1:33303516)
[chr1:33303516-chr1:33303766),"""Upstream""",[chr1:33303516-chr1:33303766)
[chr1:33303766-chr1:33304016),"""Upstream""",[chr1:33303766-chr1:33304016)


18684


2021-03-01 19:13:24 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-03-01 19:13:24 Hail: INFO: Ordering unsorted dataset with network shuffle


+---------------+------------+-------+----------+-------------------+-------------------+---------------+-------------+----------+--------+------------------------------+
| locus         | alleles    |    AC |       AF | vep.assembly_name | vep.allele_string | vep.ancestral | vep.context |  vep.end | vep.id | vep.input                    |
+---------------+------------+-------+----------+-------------------+-------------------+---------------+-------------+----------+--------+------------------------------+
| locus<GRCh38> | array<str> | int32 |  float64 | str               | str               | str           | str         |    int32 | str    | str                          |
+---------------+------------+-------+----------+-------------------+-------------------+---------------+-------------+----------+--------+------------------------------+
| chr1:50882553 | ["G","T"]  |    28 | 4.12e-04 | "GRCh38"          | "G/T"             | NA            | NA          | 50882553 | "."    | "chr1	50882553	.	G	T	.	.	GT" |
| chr1:50882563 | ["C","T"]  |     2 | 2.94e-05 | "GRCh38"          | "C/T"             | NA            | NA          | 50882563 | "."    | "chr1	50882563	.	C	T	.	.	GT" |
| chr1:50882566 | ["C","T"]  |     2 | 4.84e-05 | "GRCh38"          | "C/T"             | NA            | NA          | 50882566 | "."    | "chr1	50882566	.	C	T	.	.	GT" |
| chr1:50882567 | ["A","C"]  |     4 | 9.66e-05 | "GRCh38"          | "A/C"             | NA            | NA          | 50882567 | "."    | "chr1	50882567	.	A	C	.	.	GT" |
| chr1:50882584 | ["A","T"]  |     1 | 6.58e-05 | "GRCh38"          | "A/T"             | NA            | NA          | 50882584 | "."    | "chr1	50882584	.	A	T	.	.	GT" |
| chr1:50882585 | ["G","C"]  |     1 | 2.44e-05 | "GRCh38"          | "G/C"             | NA            | NA          | 50882585 | "."    | "chr1	50882585	.	G	C	.	.	GT" |
| chr1:50882588 | ["C","A"]  |     2 | 1.34e-04 | "GRCh38"          | "C/A"             | NA            | NA          | 50882588 | "."    | "chr1	50882588	.	C	A	.	.	GT" |
| chr1:50882591 | ["G","C"]  |     1 | 1.48e-05 | "GRCh38"          | "G/C"             | NA            | NA          | 50882591 | "."    | "chr1	50882591	.	G	C	.	.	GT" |
| chr1:50882592 | ["A","T"]  |     1 | 1.48e-05 | "GRCh38"          | "A/T"             | NA            | NA          | 50882592 | "."    | "chr1	50882592	.	A	T	.	.	GT" |
| chr1:50882595 | ["C","G"]  |     2 | 3.99e-04 | "GRCh38"          | "C/G"             | NA            | NA          | 50882595 | "."    | "chr1	50882595	.	C	G	.	.	GT" |
+---------------+------------+-------+----------+-------------------+-------------------+---------------+-------------+----------+--------+------------------------------+

+---------------------------------------------------------------------------------------------------------------------+-----------------------------+
| vep.intergenic_consequences                                                                                         | vep.most_severe_consequence |
+---------------------------------------------------------------------------------------------------------------------+-----------------------------+
| array<struct{allele_num: int32, consequence_terms: array<str>, impact: str, minimised: int32, variant_allele: str}> | str                         |
+---------------------------------------------------------------------------------------------------------------------+-----------------------------+
| NA                                                                                                                  | "intron_variant"            |
| NA                                                                                                                  | "intron_variant"            |
| NA                                                                                                                  | "intron_variant"            |
| NA                                                           

In [ ]:
merge_biomart_refseq_38 = hl.import_table("/gstock/EXOTIC/data/GENOMICS/merge_biomart_refseq_38.csv", impute=True, delimiter='\t')
merge_biomart_refseq_38.show()

In [ ]:
data_lite = data_lite.annotate(biomart = merge_biomart_refseq_38[data_lite.].)


In [ ]:
import pandas as pd
# bed = pd.DataFrame([['chr1', 'chr11'], [10031,66510646],[10032,66510647]]).T
bed = pd.DataFrame([['chr1'], [10031],[10031]]).T
# bed = pd.DataFrame([['chr11'], [66510646],[66510647]]).T
bed.to_csv('test2.bed', sep='\t', header=False, index=False)
interval_table = hl.import_bed('test2.bed', reference_genome='GRCh38')
interval_table

In [ ]:
gnomad.utils.filtering.filter_to_autosomes(test)

In [ ]:
test.show()

In [ ]:
data_lite.filter(data_lite.popmax.AF > 1e-2).popmax.show()

In [ ]:

ds_result.show()

In [ ]:
interval = hl.parse_locus_interval('chr11:66510646-66510647', reference_genome='GRCh38')
data_lite = data.filter(interval.contains(data.locus))
data_lite.show()

In [ ]:
filtered_mt = data_lite.filter(hl.is_defined(interval_table[data_lite.locus]))
filtered_mt.show()

In [ ]:
filtered_mt.filter(filtered_mt.popmax.AF > 2.5e-5).to_pandas()